In [13]:
from keybert import KeyBERT
from konlpy.tag import Kkma

# KeyBERT 모델과 Kkma 형태소 분석기 초기화
kw_model = KeyBERT(model='all-mpnet-base-v2')
kkma = Kkma()

# 키워드 추출 함수
def extract_keywords(text, top_n=10, keyphrase_ngram_range=(1, 1), stop_words=None):
    # KeyBERT를 사용하여 키워드 추출
    keywords = kw_model.extract_keywords(text, 
                                         keyphrase_ngram_range=keyphrase_ngram_range, 
                                         stop_words=stop_words, 
                                         top_n=top_n)
    
    keywords_with_lemma = {}
    for keyword, score in keywords:
        # 형태소 분석 및 명사, 동사, 형용사만 추출
        lemmas = kkma.pos(keyword, join=True)
        filtered_lemmas = [lemma.split('/')[0] for lemma in lemmas if lemma.split('/')[1] in ['NNG', 'NNP', 'VA', 'VV']]
        
        # 중복된 키워드를 제거하고, 최고 점수만 유지
        for lemma in filtered_lemmas:
            if lemma not in keywords_with_lemma or score > keywords_with_lemma[lemma]:
                keywords_with_lemma[lemma] = score

    # 추출된 키워드와 점수를 정렬하여 반환
    sorted_keywords = sorted(keywords_with_lemma.items(), key=lambda x: x[1], reverse=True)
    return sorted_keywords

text = """
#Person1#: 실례합니다, 열쇠 한 묶음 보셨나요?
#Person2#: 어떤 종류의 열쇠인가요?
#Person1#: 5개의 열쇠와 작은 발 장식이 있어요.
#Person2#: 안타깝네요! 저는 보지 못했습니다.
#Person1#: 그럼, 찾는 데 도와주실 수 있나요? 제가 여기는 처음이라서요.
#Peson2#: 물론입니다. 기꺼이 도와드리겠습니다. 사라진 열쇠를 찾는 데 도와드리겠습니다.
#Person1#: 정말 친절하시네요.
#Person2#: 별 말씀을요. 어, 찾았어요.
#Person1#: 오, 하느님 감사합니다! 어떻게 감사의 말씀을 드려야 할지 모르겠네요.
#Person2#: 천만에요.
"""

keywords = extract_keywords(text, top_n=10)
keywords_string = '"' + ', '.join([f"{keyword}" for keyword, score in keywords]) + '"'
print(keywords_string)


"물론, 말씀, 요, 천만, 감사, 친절, 처음, 도와, 주실, 사라지"


In [14]:
import pandas as pd

def generate_summary(row):
    text = row['dialogue']
    keywords = extract_keywords(text, top_n=10)
    summary = ', '.join([f"{keyword}" for keyword, score in keywords])
    return summary

df = pd.read_csv('/kkh/data/test.csv')
df['summary'] = df.apply(generate_summary, axis=1)
df = df.drop(columns=['dialogue'])
df.to_csv('/kkh/data/kkh-submit.csv', index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'data/test.csv'